In [1]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00226/OpportunityUCIDataset.zip -P ../data/raw/

'wget' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


In [2]:
!mkdir ../data/processed
!python3 ../src/data/preprocess_data.py -i  ../data/raw/OpportunityUCIDataset.zip -o ../processed/oppChallenge_gestures.data

命令语法不正确。
Traceback (most recent call last):
  File "../src/data/preprocess_data.py", line 10, in <module>
    from pandas import Series
ModuleNotFoundError: No module named 'pandas'


### Setup

In [3]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import _pickle as cp
import matplotlib.pyplot as plt
import pandas as pd
import sklearn.metrics as metrics
from sklearn.metrics import plot_confusion_matrix

import torch
from torch import nn
import torch.nn.functional as F

from sklearn.metrics import confusion_matrix
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from tensorboardX import SummaryWriter

import netron

#NB_SENSOR_CHANNELS = 113
NB_SENSOR_CHANNELS = 15
SLIDING_WINDOW_LENGTH = 40
SLIDING_WINDOW_STEP = 20
NB_CLASS = 16

In [4]:
import os
import sys

# add the 'src' directory as one where we can import modules
src_dir = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)

from data.sliding_window import sliding_window

## Load in Data

In [5]:

def load_dataset(filename):
    
    with open(filename, 'rb') as f:
        data = cp.load(f)
    
    X_train, y_train = data[0]
    X_test, y_test = data[1]

    print(" ..from file {}".format(filename))
    print(" ..reading instances: train {0}, test {1}".format(X_train.shape, X_test.shape))

    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)
    
    print("..part of x_train",type(X_train))
    #print(X_train[:23,:2])
    # The targets are casted to int8 for GPU compatibility.
    y_train = y_train.astype(np.uint8)
    y_test = y_test.astype(np.uint8)

    return X_train, y_train, X_test, y_test

print("Loading data...")
#X_train, y_train, X_test, y_test = load_dataset('../data/processed/oppChallenge_gestures.data')
X_train, y_train, X_test, y_test = load_dataset('C:\\Users\\Zhiha\\Documents\\file_only\\research\\tool_program\\ori\\Sensor-Based-Human-Activity-Recognition-DeepConvLSTM-Pytorch\\data\\processed\\mygestures_ww.data')


Loading data...
 ..from file C:\Users\Zhiha\Documents\file_only\research\tool_program\ori\Sensor-Based-Human-Activity-Recognition-DeepConvLSTM-Pytorch\data\processed\mygestures_ww.data
 ..reading instances: train (163388, 15), test (41672, 15)
..part of x_train <class 'numpy.ndarray'>


## Data Segmentation

In [6]:
assert NB_SENSOR_CHANNELS == X_train.shape[1]
def opp_sliding_window(data_x, data_y, ws, ss):
    data_x = sliding_window(data_x, (ws, data_x.shape[1]), (ss, 1))
    data_y = np.asarray([[i[-1]] for i in sliding_window(data_y, ws, ss)])
    return data_x.astype(np.float32), data_y.reshape(len(data_y)).astype(np.uint8)

# Sensor data is segmented using a sliding window mechanism
X_train, y_train = opp_sliding_window(X_train, y_train, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
X_test, y_test = opp_sliding_window(X_test, y_test, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
#print(X_train[:24,:3])
print(X_train.shape)
# Data is reshaped
X_train = X_train.reshape((-1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS)) # for input to Conv1D
X_test = X_test.reshape((-1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS)) # for input to Conv1D

#print(y_train[10:25,])
print(" ..after sliding and reshaping, train data: inputs {0}, targets {1}".format(X_train.shape, y_train.shape))
print(" ..after sliding and reshaping, test data : inputs {0}, targets {1}".format(X_test.shape, y_test.shape))

(8168, 40, 15)
 ..after sliding and reshaping, train data: inputs (8168, 40, 15), targets (8168,)
 ..after sliding and reshaping, test data : inputs (2082, 40, 15), targets (2082,)


## Get labels

In [7]:
#='D:\\test\\project2files\\2\\files\\labels.csv'
def return_labels(f,series):
    ss = []
    df = pd.read_csv(f,header=None)
    state = df.iloc[1:,0].to_numpy()
    activities = df.iloc[0,1:].to_numpy()
    state_len = len(state)
    act_len = len(activities)
    for s in series:        
        x = s % act_len #acts
        y = s // act_len #states
        #print(state[y]+'+'+activities[x])
        ss.append(str(state[y])+'+'+str(activities[x]))
    return ss,activities

## draw confusion matrix

In [8]:
''''''
def plot_confusion_matrix(cm,t,title='Confusion Matrix', cmap=plt.cm.binary,):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    if t == 1:
        xlocations = np.array(range(NB_CLASS))
    elif t == 2:
        print("is type 2")
        xlocations = np.array(range(int(NB_CLASS/2)))
    plt.xticks(xlocations, xlocations, rotation=90)
    plt.yticks(xlocations, xlocations)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    

    
def print_cm(cm,epoch,_t):
    np.set_printoptions(precision=2)
    cm_normalized = 100*cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print (cm_normalized)
    plt.figure(figsize=(12, 8), dpi=120)
    ###
    
    if _t == 1:
        ind_array = np.arange(NB_CLASS)
        tick_marks = np.array(range(NB_CLASS)) + 0.5
    elif _t == 2:
        ind_array = np.arange(int(NB_CLASS/2))
        tick_marks = np.array(range(int(NB_CLASS/2))) + 0.5
        
    x, y = np.meshgrid(ind_array, ind_array)

    for x_val, y_val in zip(x.flatten(), y.flatten()):
        c = cm_normalized[y_val][x_val]
        if c > 0.01:
            #plt.text(x_val, y_val, "%0.2f" % (c,), color='red', fontsize=12, va='center', ha='center')
            plt.text(x_val, y_val, "%3.1f" %(c,), color='red', fontsize=12, va='center', ha='center')
    # offset the tick
    plt.gca().set_xticks(tick_marks, minor=True)
    plt.gca().set_yticks(tick_marks, minor=True)
    plt.gca().xaxis.set_ticks_position('none')
    plt.gca().yaxis.set_ticks_position('none')
    plt.grid(True, which='minor', linestyle='-')
    plt.gcf().subplots_adjust(bottom=0.15)

    
    # show confusion matrix
    if _t == 1:
        plot_confusion_matrix(cm_normalized,_t, title='Normalized confusion matrix')
        plt.savefig(os.path.abspath(os.path.join(os.getcwd(), ".."))+ '/cm_results/'+'t_matrix'+str(epoch)+'.png', format='png')
    elif _t == 2:
        plot_confusion_matrix(cm_normalized,_t,title='Normalized confusion matrix')
        plt.savefig(os.path.abspath(os.path.join(os.getcwd(), ".."))+ '/cm_results/'+'a_matrix'+str(epoch)+'.png', format='png')
    plt.close()
    #provide report


## Define the Model

In [9]:
class HARModel(nn.Module):
    
    def __init__(self, n_hidden=128, n_layers=1, n_filters=64, 
                 n_classes=NB_CLASS, filter_size=3, drop_prob=0.5):
        super(HARModel, self).__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.n_filters = n_filters
        self.n_classes = n_classes
        self.filter_size = filter_size
             
        self.conv1 = nn.Conv1d(NB_SENSOR_CHANNELS, n_filters, filter_size)
        self.conv2 = nn.Conv1d(n_filters, n_filters, filter_size)
        self.conv3 = nn.Conv1d(n_filters, n_filters, filter_size)
        self.conv4 = nn.Conv1d(n_filters, n_filters, filter_size)
        
        self.lstm1  = nn.LSTM(n_filters, n_hidden, n_layers)
        self.lstm2  = nn.LSTM(n_hidden, n_hidden, n_layers)
        
        self.fc = nn.Linear(n_hidden, n_classes)

        self.dropout = nn.Dropout(drop_prob)
    
    def forward(self, x, hidden, batch_size):
        #print(x.size())
        
        #x = x.view(-1, NB_SENSOR_CHANNELS, SLIDING_WINDOW_LENGTH)#reshape
        x = x.reshape(-1, NB_SENSOR_CHANNELS, SLIDING_WINDOW_LENGTH)#reshape
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        m = x.size()[2]
        x = x.view(m, -1, self.n_filters)
        x, hidden = self.lstm1(x, hidden)
        x, hidden = self.lstm2(x, hidden)
        
        x = x.contiguous().view(-1, self.n_hidden)
        x = self.dropout(x)
        x = self.fc(x)
        
        out = x.view(batch_size, -1, self.n_classes)[:,-1,:]
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        '''
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        '''
        hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
    
net = HARModel()

## Initialize Model Weights

In [10]:
def init_weights(m):
    if type(m) == nn.LSTM:
        for name, param in m.named_parameters():
            if 'weight_ih' in name:
                torch.nn.init.orthogonal_(param.data)
            elif 'weight_hh' in name:
                torch.nn.init.orthogonal_(param.data)
            elif 'bias' in name:
                param.data.fill_(0)
    elif type(m) == nn.Conv1d or type(m) == nn.Linear:
        torch.nn.init.orthogonal_(m.weight)
        m.bias.data.fill_(0)
net.apply(init_weights)    

HARModel(
  (conv1): Conv1d(15, 64, kernel_size=(3,), stride=(1,))
  (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
  (conv3): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
  (conv4): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
  (lstm1): LSTM(64, 128)
  (lstm2): LSTM(128, 128)
  (fc): Linear(in_features=128, out_features=16, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

## Generate Minibatches

In [11]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=True):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [12]:
## check if GPU is available
#train_on_gpu = torch.cuda.is_available()
#if(train_on_gpu):
    #print('Training on GPU!')
#else: 
    #print('No GPU available, training on CPU; consider making n_epochs very small.')

## Train the Network

In [13]:

def train(net, epochs=50, batch_size=16, lr=0.01):
    min_loss = 100
    max_acc = 0.1
    opt = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=1e-3)
    criterion = nn.CrossEntropyLoss()
    
    #if(train_on_gpu):
        #net.cuda()
    valloss =[]
    valacc = []
    traloss =[]
    t = np.arange(NB_CLASS)
    y_unique,acts = return_labels('D:\\test\\project2files\\2\\test\\files\\labels.csv',t)
    print(type(acts))
    for e in range(epochs):
        
        # initialize hidden state
        h = net.init_hidden(batch_size)         
        train_losses = []    
        net.train()
        for batch in iterate_minibatches(X_train, y_train, batch_size):
            x, y = batch

            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            

            #if(train_on_gpu):
            #        inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])
            
            # zero accumulated gradients
            opt.zero_grad()   
            
            # get the output from the model
            output, h = net(inputs, h, batch_size)
            
            loss = criterion(output, targets.long())
            train_losses.append(loss.item())
            loss.backward()
            opt.step()
            
        val_h = net.init_hidden(batch_size)
        val_losses = []
        ##
        val_output = []
        val_target = []
        ##
        accuracy=0
        f1score=0
        net.eval()
        with torch.no_grad():
            for batch in iterate_minibatches(X_test, y_test, batch_size):
                x, y = batch     
                val_target.extend(y)
                inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
                #print("targets: \n",targets)
                #val_target.extend(targets)
                

                val_h = tuple([each.data for each in val_h])

                #if(train_on_gpu):
                #    inputs, targets = inputs.cuda(), targets.cuda()
                    
                output, val_h= net(inputs, val_h, batch_size)
                #print("output: \n",output)
                #val_output.extend(output)

                val_loss = criterion(output, targets.long())
                val_losses.append(val_loss.item())

                top_p, top_class = output.topk(1, dim=1)#get the possible class prediction
                
                equals = top_class == targets.view(*top_class.shape).long()
                #print("top_class: \n",top_class.view(*targets.shape))
                val_output.extend(top_class.view(*targets.shape))
                accuracy += torch.mean(equals.type(torch.FloatTensor))
                f1score += metrics.f1_score(top_class.cpu(), targets.view(*top_class.shape).long().cpu(), average='weighted')
            
        net.train() # reset to train mode after iterationg through validation data
        #print(batch_size)        
        print("Epoch: {}/{}...".format(e+1, epochs),
        "Train Loss: {:.4f}...".format(np.mean(train_losses)),
        "Val Loss: {:.4f}...".format(np.mean(val_losses)),
        "Val Acc: {:.4f}...".format(accuracy/(len(X_test)//batch_size)),
        "F1-Score: {:.4f}...".format(f1score/(len(X_test)//batch_size)))
        print("Length of true and predict:{}/{}...".format(len(val_target),len(val_output)))
        cm = confusion_matrix(val_target,val_output)
        val_target2 = [x-NB_CLASS/2 if x>=(NB_CLASS/2) else x for x in val_target]
        val_output2 = [x-NB_CLASS/2 if x>=(NB_CLASS/2) else x for x in val_output]
        cm2 = confusion_matrix(val_target2,val_output2)
        print("original:\n",classification_report(val_target,val_output,target_names = np.array(y_unique)))
        print("conludes:\n",classification_report(val_target2,val_output2,target_names = list(map(str,acts))))
        
        tl = np.mean(train_losses)
        vl = np.mean(val_losses)
        ac = accuracy/(len(X_test)//batch_size)
        
        
        min_loss = vl
        max_acc = ac
        print("save model")
        #torch.save(net.state_dict(),os.path.abspath(os.path.join(os.getcwd(), "..","cm_results","model"+str(e)+".pth")))
        print_cm(cm,e,1)
        print_cm(cm2,e,2)
        
            
        valloss.append(vl)
        valacc.append(ac)
        traloss.append(tl)
    x = np.arange(epochs)
    
    plt.cla()
    plt.gca()
    plt.plot(x,valloss,color='green',label='val_loss')
    plt.plot(x,traloss,color='red',label='train_loss')
    plt.plot(x,valacc,color='blue',label='val_acc')
    plt.xlabel('epochs')
    plt.savefig(os.path.abspath(os.path.join(os.getcwd(), "..","cm_results","result_t.jpg")))
    plt.close()
    
    return inputs, val_h, batch_size
    
inputs, val_h, batch_size = train(net)

#writer = SummaryWriter('runs/fashion_mnist_experiment_1')
#tb = SummaryWriter()

#tb.add_graph(net,(inputs, val_h, torch.tensor(batch_size)))
#tb.close()
'''
onnxpath = os.path.abspath(os.path.join(os.getcwd(), "..","cm_results","onnx_model.onnx"))
torch.onnx.export(net,(inputs, val_h, torch.tensor(batch_size)),onnxpath)
netron.start(onnxpath)
'''



<class 'numpy.ndarray'>
Epoch: 1/50... Train Loss: 2.6912... Val Loss: 2.6604... Val Acc: 0.1433... F1-Score: 0.2391...
Length of true and predict:2080/2080...
original:
                            precision    recall  f1-score   support

    standing+BackandForth       0.00      0.00      0.00        97
       standing+Clockwise       0.00      0.00      0.00        75
standing+CounterClockwise       0.00      0.00      0.00        71
            standing+down       0.00      0.00      0.00        83
            standing+left       0.00      0.00      0.00       147
             standing+nan       0.00      0.00      0.00       327
           standing+right       0.00      0.00      0.00        72
              standing+up       0.00      0.00      0.00       116
     walking+BackandForth       0.00      0.00      0.00        94
        walking+Clockwise       0.00      0.00      0.00        99
 walking+CounterClockwise       0.00      0.00      0.00       114
             walking+dow

                   precision    recall  f1-score   support

    BackandForth       0.00      0.00      0.00       191
       Clockwise       0.00      0.00      0.00       174
CounterClockwise       0.00      0.00      0.00       185
            down       0.00      0.00      0.00       202
            left       0.00      0.00      0.00       251
             nan       0.30      1.00      0.46       626
           right       0.00      0.00      0.00       211
              up       0.00      0.00      0.00       240

        accuracy                           0.30      2080
       macro avg       0.04      0.12      0.06      2080
    weighted avg       0.09      0.30      0.14      2080

save model
[[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 100.
    0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 100.
    0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 100.
    0.   0.]
 [  0.   0.   0.   0.   0.   0.   

[[  0.   0.   0.   0.   0. 100.   0.   0.]
 [  0.   0.   0.   0.   0. 100.   0.   0.]
 [  0.   0.   0.   0.   0. 100.   0.   0.]
 [  0.   0.   0.   0.   0. 100.   0.   0.]
 [  0.   0.   0.   0.   0. 100.   0.   0.]
 [  0.   0.   0.   0.   0. 100.   0.   0.]
 [  0.   0.   0.   0.   0. 100.   0.   0.]
 [  0.   0.   0.   0.   0. 100.   0.   0.]]
is type 2
Epoch: 6/50... Train Loss: 2.6823... Val Loss: 2.6558... Val Acc: 0.1433... F1-Score: 0.2418...
Length of true and predict:2080/2080...
original:
                            precision    recall  f1-score   support

    standing+BackandForth       0.00      0.00      0.00        97
       standing+Clockwise       0.00      0.00      0.00        75
standing+CounterClockwise       0.00      0.00      0.00        71
            standing+down       0.00      0.00      0.00        83
            standing+left       0.00      0.00      0.00       147
             standing+nan       0.00      0.00      0.00       329
           standing+right   

Epoch: 8/50... Train Loss: 2.6807... Val Loss: 2.6620... Val Acc: 0.1433... F1-Score: 0.2420...
Length of true and predict:2080/2080...
original:
                            precision    recall  f1-score   support

    standing+BackandForth       0.00      0.00      0.00        97
       standing+Clockwise       0.00      0.00      0.00        75
standing+CounterClockwise       0.00      0.00      0.00        71
            standing+down       0.00      0.00      0.00        83
            standing+left       0.00      0.00      0.00       147
             standing+nan       0.00      0.00      0.00       329
           standing+right       0.00      0.00      0.00        72
              standing+up       0.00      0.00      0.00       116
     walking+BackandForth       0.00      0.00      0.00        94
        walking+Clockwise       0.00      0.00      0.00        99
 walking+CounterClockwise       0.00      0.00      0.00       114
             walking+down       0.00      0.00  

[[  0.   0.   0.   0.   0. 100.   0.   0.]
 [  0.   0.   0.   0.   0. 100.   0.   0.]
 [  0.   0.   0.   0.   0. 100.   0.   0.]
 [  0.   0.   0.   0.   0. 100.   0.   0.]
 [  0.   0.   0.   0.   0. 100.   0.   0.]
 [  0.   0.   0.   0.   0. 100.   0.   0.]
 [  0.   0.   0.   0.   0. 100.   0.   0.]
 [  0.   0.   0.   0.   0. 100.   0.   0.]]
is type 2
Epoch: 11/50... Train Loss: 2.6717... Val Loss: 2.6334... Val Acc: 0.1572... F1-Score: 0.2502...
Length of true and predict:2080/2080...
original:
                            precision    recall  f1-score   support

    standing+BackandForth       0.00      0.00      0.00        97
       standing+Clockwise       0.00      0.00      0.00        75
standing+CounterClockwise       0.00      0.00      0.00        71
            standing+down       0.00      0.00      0.00        82
            standing+left       0.00      0.00      0.00       147
             standing+nan       0.91      0.09      0.16       329
           standing+right  

...

is type 2
Epoch: 45/50... Train Loss: 0.4254... Val Loss: 0.5661... Val Acc: 0.8120... F1-Score: 0.8213...
Length of true and predict:2080/2080...
original:
                            precision    recall  f1-score   support

    standing+BackandForth       0.68      0.74      0.71        97
       standing+Clockwise       0.83      0.60      0.70        75
standing+CounterClockwise       0.74      0.75      0.74        71
            standing+down       0.94      0.61      0.74        83
            standing+left       0.88      0.95      0.91       147
             standing+nan       0.87      0.93      0.90       328
           standing+right       0.72      0.51      0.60        71
              standing+up       0.90      0.33      0.48       116
     walking+BackandForth       0.78      0.83      0.80        94
        walking+Clockwise       0.73      0.88      0.80        99
 walking+CounterClockwise       0.96      0.91      0.94       114
             walking+down       0.76 

is type 2
Epoch: 47/50... Train Loss: 0.4029... Val Loss: 0.6170... Val Acc: 0.8058... F1-Score: 0.8124...
Length of true and predict:2080/2080...
original:
                            precision    recall  f1-score   support

    standing+BackandForth       0.70      0.70      0.70        97
       standing+Clockwise       0.81      0.45      0.58        75
standing+CounterClockwise       0.87      0.75      0.80        71
            standing+down       0.70      0.82      0.76        83
            standing+left       0.84      0.92      0.88       147
             standing+nan       0.97      0.82      0.89       329
           standing+right       0.78      0.50      0.61        72
              standing+up       0.84      0.64      0.73       116
     walking+BackandForth       0.78      0.76      0.77        94
        walking+Clockwise       0.65      0.87      0.74        99
 walking+CounterClockwise       0.87      0.91      0.89       114
             walking+down       0.85 

is type 2
Epoch: 49/50... Train Loss: 0.3604... Val Loss: 0.5824... Val Acc: 0.8106... F1-Score: 0.8188...
Length of true and predict:2080/2080...
original:
                            precision    recall  f1-score   support

    standing+BackandForth       0.85      0.64      0.73        97
       standing+Clockwise       0.83      0.47      0.60        75
standing+CounterClockwise       0.84      0.58      0.68        71
            standing+down       0.84      0.64      0.73        83
            standing+left       0.82      0.95      0.88       146
             standing+nan       0.90      0.94      0.92       328
           standing+right       0.62      0.50      0.55        72
              standing+up       0.81      0.40      0.53       116
     walking+BackandForth       0.74      0.80      0.77        94
        walking+Clockwise       0.65      0.92      0.76        99
 walking+CounterClockwise       0.98      0.92      0.95       114
             walking+down       0.78 

is type 2


'\nonnxpath = os.path.abspath(os.path.join(os.getcwd(), "..","cm_results","onnx_model.onnx"))\ntorch.onnx.export(net,(inputs, val_h, torch.tensor(batch_size)),onnxpath)\nnetron.start(onnxpath)\n'